### Import libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_privacy
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
%matplotlib inline
tf.get_logger().setLevel('ERROR')
tf.compat.v1.disable_v2_behavior()

### Load Dataset

In [2]:
# Loading cifar10 image dataset
data_train, data_test = cifar10.load_data()
x_train, y_train = data_train
x_test, y_test = data_test

# Normalizing pixel values of images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flattening images in dataset
y_train = y_train.flatten().reshape(y_train.shape[0], 1)
y_test = y_test.flatten().reshape(y_test.shape[0], 1)

# One hot encoding of labels/target column
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print("training:", x_train.shape, y_train.shape)
print("testing:", x_test.shape, y_test.shape)

training: (50000, 32, 32, 3) (50000, 10)
testing: (10000, 32, 32, 3) (10000, 10)


### Model1: Simple CNN

In [11]:
# Hyper-parameters
epochs = 20
batch_size = 500

l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [12]:
# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [13]:
# define optimizer (dp-sgd) 
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)
# define loss function
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [14]:
# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# Train model
model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

Train on 50000 samples, validate on 10000 samples


2022-03-08 16:42:30.484258: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-08 16:42:31.402756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/20


2022-03-08 16:42:31.686466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-08 16:42:31.846137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - ETA: 0s - loss: 2.2188 - acc: 0.1664 

2022-03-08 17:04:09.816648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - 1299s 26ms/sample - loss: 2.2188 - acc: 0.1664 - val_loss: 2.1000 - val_acc: 0.2320
Epoch 2/20
50000/50000 [==============================] - 1223s 24ms/sample - loss: 2.0589 - acc: 0.2534 - val_loss: 1.9255 - val_acc: 0.3094
Epoch 3/20
50000/50000 [==============================] - 1212s 24ms/sample - loss: 1.9223 - acc: 0.3040 - val_loss: 1.8572 - val_acc: 0.3305
Epoch 4/20
50000/50000 [==============================] - 1217s 24ms/sample - loss: 1.8465 - acc: 0.3341 - val_loss: 1.8674 - val_acc: 0.3170
Epoch 5/20
50000/50000 [==============================] - 1208s 24ms/sample - loss: 1.7838 - acc: 0.3603 - val_loss: 1.7684 - val_acc: 0.3776
Epoch 6/20
50000/50000 [==============================] - 1221s 24ms/sample - loss: 1.7363 - acc: 0.3805 - val_loss: 1.7594 - val_acc: 0.3869
Epoch 7/20
50000/50000 [==============================] - 1227s 25ms/sample - loss: 1.6895 - acc: 0.4000 - val_loss: 1.7437 - val_acc: 0.3961
Epoch 8/20
50000/

In [15]:
# Calculate privacy performance
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=x_train.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=epochs,
                                              delta=1e-5)

DP-SGD with sampling rate = 1% and noise_multiplier = 1.3 iterated over 2000 steps satisfies differential privacy with eps = 1.8 and delta = 1e-05.
The optimal RDP order is 11.0.


(1.798929515280597, 11.0)

### Model2: ResNet with transfer learning

### Model3: HOG classification